## MNIST and Softmax

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
# MNIST dataset
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                         train = True,
                         transform = transforms.ToTensor(),
                         download = True)

mnist_test = dsets.MNIST(root = 'MNIST_data/',
                        train = False,
                        transform = transforms.ToTensor(),
                        download = True)

In [5]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                         batch_size = batch_size,
                                         shuffle = True,
                                         drop_last = True)

In [6]:
# MNIST data image of shape 28 * 28 = 784
linear = torch.nn.Linear(784, 10, bias = True).to(device)

In [8]:
# initialization
torch.nn.init.normal_(linear.weight)

Parameter containing:
tensor([[-0.1953, -0.4404,  0.0139,  ...,  0.7510,  1.5190,  0.9637],
        [ 0.4146, -1.0660, -0.9969,  ..., -0.3259, -0.2695, -1.3316],
        [ 0.2600, -1.1821,  0.0065,  ...,  1.0648, -1.2251,  0.9841],
        ...,
        [ 0.0279, -1.7242, -1.1723,  ...,  1.3651, -0.0689, -0.7931],
        [ 0.0952,  0.1939,  0.1636,  ..., -1.0489,  0.6224, -1.7181],
        [ 2.6504,  0.6297, -1.6571,  ..., -0.0781, -1.1398, -0.8711]],
       requires_grad=True)

In [9]:
# define cost / loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device) # Softmax is internally computed
optimizer = torch.optim.Adam(linear.parameters(), lr = learning_rate)

In [10]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        hypothesis = linear(X)
        cost = criterion(hypothesis, y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('Epoch : ', '%04d' % (epoch + 1), 'Cost = ', '{:.9f}'.format(avg_cost))

print('Learning Finished.')

Epoch :  0001 Cost =  5.707432747
Epoch :  0002 Cost =  1.566787243
Epoch :  0003 Cost =  1.038582563
Epoch :  0004 Cost =  0.828922153
Epoch :  0005 Cost =  0.710936010
Epoch :  0006 Cost =  0.633364379
Epoch :  0007 Cost =  0.578106523
Epoch :  0008 Cost =  0.535550117
Epoch :  0009 Cost =  0.502309144
Epoch :  0010 Cost =  0.476203054
Epoch :  0011 Cost =  0.454187691
Epoch :  0012 Cost =  0.435488403
Epoch :  0013 Cost =  0.420344979
Epoch :  0014 Cost =  0.406599790
Epoch :  0015 Cost =  0.394169509
Learning Finished.


In [12]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    y_test = mnist_test.test_labels.to(device)
    
    pred = linear(X_test)
    correct_pred = torch.argmax(pred, 1) == y_test
    accuracy = correct_pred.float().mean()
    print('Accuracy : ', accuracy.item())
    
    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    y_single_data = mnist_test.test_labels[r:r + 1].to(device)
    
    print('Label : ', y_single_data.item())
    single_pred = linear(X_single_data)
    print('Prediction : ', torch.argmax(single_pred, 1).item())

Accuracy :  0.8956000208854675
Label :  5
Prediction :  5
